# Primer Punto

Entrene un modelo CART o C4.5/ID3 para obtener un árbol de decisión y generar reglas explı́citas de clasiﬁcación. A continuación, analice las reglas obtenidas, identiﬁque cuáles resultan más útiles (por ejemplo, por pureza, cobertura y simplicidad), y realice una validación cruzada del modelo.